## Performance debugging using Profiler

In [1]:
import torch
import numpy as np
from torch import nn
import torch.autograd.profiler as profiler

In [2]:
class MyModule(nn.Module):
    def __init__(self, in_features: int, out_features: int, bias: bool = True):
        super(MyModule, self).__init__()
        self.linear = nn.Linear(in_features, out_features, bias)

    def forward(self, image, mask):
        
        with profiler.record_function("LINEAR PASS"):
            out = self.linear(image)

        with profiler.record_function("MASK INDICES"):
            threshold = out.sum(axis=1).mean().item()
            hi_idx = np.argwhere(mask.cpu().numpy() > threshold)
            hi_idx = torch.from_numpy(hi_idx).cuda()

        return out, hi_idx

### Profile the forward pass
Before we run the profiler, we warm-up CUDA to ensure accurate performance benchmarking. We wrap the forward pass of our module in the profiler.profile context manager. The with_stack=True parameter appends the file and line number of the operation in the trace.

`with_stack=True` incurs an additional overhead, and is better suited for investigating code. Remember to remove it if you are benchmarking performance.

In [3]:
model = MyModule(500, 10).cuda()
image = torch.rand(128, 500).cuda()
mask = torch.rand((500, 500, 500), dtype=torch.double).cuda()

# warm-up
model(image, mask)

with profiler.profile(with_stack=True, profile_memory=True) as prof:
    out, idx = model(image, mask)

### Print profiler results
Finally, we print the profiler results. profiler.key_averages aggregates the results by operator name, and optionally by input shapes and/or stack trace events. Grouping by input shapes is useful to identify which tensor shapes are utilized by the model.

In [4]:
print(prof.key_averages(group_by_stack_n=5).table(sort_by='self_cpu_time_total', row_limit=5))

-----------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ---------------------------------------------------------------------------  
                         Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem      CUDA Mem  Self CUDA Mem    # of Calls  Source Location                                                              
-----------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ---------------------------------------------------------------------------  
                 MASK INDICES        70.91%     450.747ms        99.79%     634.354ms     634.354ms          -4 b    -953.67 Mb      74.00 Mb      -1.00 Kb             1  ...\envs\pro_1\lib\site-packages\torch\autograd\profiler.py(616): __enter__  
   

In [5]:
model = MyModule(500, 10).cuda()
image = torch.rand(128, 500).cuda()
mask = torch.rand((500, 500, 500), dtype=torch.float).cuda()

# warm-up
model(image, mask)

with profiler.profile(with_stack=True, profile_memory=True) as prof:
    out, idx = model(image, mask)

print(prof.key_averages(group_by_stack_n=5).table(sort_by='self_cpu_time_total', row_limit=5))

-----------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ---------------------------------------------------------------------------  
                         Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem      CUDA Mem  Self CUDA Mem    # of Calls  Source Location                                                              
-----------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ---------------------------------------------------------------------------  
                 MASK INDICES        81.40%        2.113s        99.95%        2.594s        2.594s          -4 b    -476.84 Mb       2.79 Gb      -1.00 Kb             1  ...\envs\pro_1\lib\site-packages\torch\autograd\profiler.py(616): __enter__  
   

### Improve time performance
While the time consumed has also reduced a bit, it’s still too high. Turns out copying a matrix from CUDA to CPU is pretty expensive! The `aten::copy_` operator in `forward (12)` copies `mask` to CPU so that it can use the NumPy `argwhere` function. `aten::copy_` at `forward(13)` copies the array back to CUDA as a tensor. We could eliminate both of these if we use a `torch` function `nonzero()` here instead.

In [6]:
class MyModule(nn.Module):
    def __init__(self, in_features: int, out_features: int, bias: bool = True):
        super(MyModule, self).__init__()
        self.linear = nn.Linear(in_features, out_features, bias)

    def forward(self, input, mask):
        with profiler.record_function("LINEAR PASS"):
            out = self.linear(input)

        with profiler.record_function("MASK INDICES"):
            threshold = out.sum(axis=1).mean()
            hi_idx = (mask > threshold).nonzero(as_tuple=True)

        return out, hi_idx


model = MyModule(500, 10).cuda()
input = torch.rand(128, 500).cuda()
mask = torch.rand((500, 500, 500), dtype=torch.float).cuda()

# warm-up
model(input, mask)

with profiler.profile(with_stack=True, profile_memory=True) as prof:
    out, idx = model(input, mask)

print(prof.key_averages(group_by_stack_n=5).table(sort_by='self_cpu_time_total', row_limit=5))

-----------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ---------------------------------------------------------------------------  
                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem      CUDA Mem  Self CUDA Mem    # of Calls  Source Location                                                              
-----------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ---------------------------------------------------------------------------  
          aten::nonzero        83.90%      17.940ms        85.40%      18.262ms      18.262ms           0 b           0 b       2.16 Gb           0 b             1  <ipython-input-6-c7eb287ee4a0>(12): forward                                  
                           